## Introduction

[![Binder](https://notebooks.gesis.org/binder/badge_logo.svg)](https://notebooks.gesis.org/binder/v2/gh/imperial-genomics-facility/scanpy-notebook-image/v2?urlpath=lab/tree/examples/Case-study_Mouse-intestinal-epithelium.ipynb)

This notebook for running single cell data analysis (for a multiple samples) using the following packages:

* [Scanpy](https://scanpy-tutorials.readthedocs.io/en/latest): Single cell RNA-Seq data processing
* [scran](https://bioconductor.org/packages/3.11/bioc/html/scran.html): Preprocessing and normalization
* [slignshot](https://bioconductor.org/packages/release/bioc/html/slingshot.html): Pseudotime and trajectory inference


We took the codes and documentation from [Best practices in single-cell RNA-seq analysis: a tutorial](https://github.com/theislab/single-cell-tutorial) for building this example notebook and used the dataset ([GSE92332](https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSE92332)) which includes samples from different regions of the mouse intestinal epithelium ([Haber et al. 2018](https://www.ncbi.nlm.nih.gov/pubmed/29144463)).

Due to memory (RAM) limitation, we have used only 3 samples and included only few part of the analysis from the original source.

##  Loading the libraries

We need to load all the required libraries to environment before we can run any of the analysis steps. Also, we are checking the version information for most of the major packages used for analysis.

In [ ]:
import scanpy as sc
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib import colors
import seaborn as sb
from gprofiler import GProfiler

import rpy2.rinterface_lib.callbacks
import logging

from rpy2.robjects import pandas2ri
import anndata2ri

In [ ]:
# Ignore R warning messages
#Note: this can be commented out to get more verbose R output
rpy2.rinterface_lib.callbacks.logger.setLevel(logging.ERROR)

# Automatically convert rpy2 outputs to pandas dataframes
pandas2ri.activate()
anndata2ri.activate()
%load_ext rpy2.ipython

plt.rcParams['figure.figsize']=(8,8) #rescale figures
sc.settings.verbosity = 3
#sc.set_figure_params(dpi=200, dpi_save=300)
sc.logging.print_versions()

In [ ]:
%%R
# Load libraries from correct lib Paths for my environment - ignore this!
#.libPaths(.libPaths()[c(3,2,1)])

# Load all the R libraries we will be using in the notebook
library(scran)
library(RColorBrewer)
library(slingshot)
library(monocle)
library(gam)
library(clusterExperiment)
library(ggplot2)
library(plyr)
library(MAST)

## Fetch raw data from NCBI GEO

The following steps are only required for downloading test data from NCBI GEO.

In [ ]:
!rm -rf /tmp/GSE92332_RAW;mkdir /tmp/GSE92332_RAW
!cd /tmp/GSE92332_RAW;\
wget -q -O /tmp/GSE92332_RAW/GSE92332_RAW.tar  \
  http://ftp.ncbi.nlm.nih.gov/geo/series/GSE92nnn/GSE92332/suppl/GSE92332_RAW.tar
!cd /tmp/GSE92332_RAW;tar -xf GSE92332_RAW.tar

## Reading in the data

Scanpy expects the data to be stored in the format cells x genes, so we need to transpose the data matrix.

In [ ]:
# Set up data loading, we are using only single dataset for each type of cell type due to memory limitation

#Data files
sample_strings = ['Duo_M1', 'Jej_M1',  'Il_M1']
sample_id_strings = ['3', '5','7']
file_base = '/tmp/GSE92332_RAW/GSM283657'
exp_string = '_Regional_'
data_file_end = '_matrix.mtx.gz'
barcode_file_end = '_barcodes.tsv.gz'
gene_file_end = '_genes.tsv.gz'
cc_genes_file = '/tmp/Macosko_cell_cycle_genes.txt'

In [ ]:
# First data set load & annotation
#Parse Filenames
sample = sample_strings.pop(0)
sample_id = sample_id_strings.pop(0)
data_file = file_base+sample_id+exp_string+sample+data_file_end
barcode_file = file_base+sample_id+exp_string+sample+barcode_file_end
gene_file = file_base+sample_id+exp_string+sample+gene_file_end

#Load data
adata = sc.read(data_file, cache=True)
adata = adata.transpose()
adata.X = adata.X.toarray()

barcodes = pd.read_csv(barcode_file, header=None, sep='\t')
genes = pd.read_csv(gene_file, header=None, sep='\t')

#Annotate data
barcodes.rename(columns={0:'barcode'}, inplace=True)
barcodes.set_index('barcode', inplace=True)
adata.obs = barcodes
adata.obs['sample'] = [sample]*adata.n_obs
adata.obs['region'] = [sample.split("_")[0]]*adata.n_obs
adata.obs['donor'] = [sample.split("_")[1]]*adata.n_obs

genes.rename(columns={0:'gene_id', 1:'gene_symbol'}, inplace=True)
genes.set_index('gene_symbol', inplace=True)
adata.var = genes

In [ ]:
# Loop to load rest of data sets
for i in range(len(sample_strings)):
    #Parse Filenames
    sample = sample_strings[i]
    sample_id = sample_id_strings[i]
    data_file = file_base+sample_id+exp_string+sample+data_file_end
    barcode_file = file_base+sample_id+exp_string+sample+barcode_file_end
    gene_file = file_base+sample_id+exp_string+sample+gene_file_end
    
    #Load data
    adata_tmp = sc.read(data_file, cache=True)
    adata_tmp = adata_tmp.transpose()
    adata_tmp.X = adata_tmp.X.toarray()

    barcodes_tmp = pd.read_csv(barcode_file, header=None, sep='\t')
    genes_tmp = pd.read_csv(gene_file, header=None, sep='\t')
    
    #Annotate data
    barcodes_tmp.rename(columns={0:'barcode'}, inplace=True)
    barcodes_tmp.set_index('barcode', inplace=True)
    adata_tmp.obs = barcodes_tmp
    adata_tmp.obs['sample'] = [sample]*adata_tmp.n_obs
    adata_tmp.obs['region'] = [sample.split("_")[0]]*adata_tmp.n_obs
    adata_tmp.obs['donor'] = [sample.split("_")[1]]*adata_tmp.n_obs
    
    genes_tmp.rename(columns={0:'gene_id', 1:'gene_symbol'}, inplace=True)
    genes_tmp.set_index('gene_symbol', inplace=True)
    adata_tmp.var = genes_tmp
    adata_tmp.var_names_make_unique()

    # Concatenate to main adata object
    adata = adata.concatenate(adata_tmp, batch_key='sample_id')
    #adata.var['gene_id'] = adata.var['gene_id-1']
    #adata.var.drop(columns = ['gene_id-1', 'gene_id-0'], inplace=True)
    adata.obs.drop(columns=['sample_id'], inplace=True)
    adata.obs_names = [c.split("-")[0] for c in adata.obs_names]
    adata.obs_names_make_unique(join='_')

In [ ]:
#Assign variable names and gene id columns
adata.var_names = [g.split("_")[1] for g in adata.var_names]
adata.var['gene_id'] = [g.split("_")[1] for g in adata.var['gene_id']]

In [ ]:
# Annotate the data sets
print(adata.obs['region'].value_counts())
print('')
print(adata.obs['donor'].value_counts())
print('')
print(adata.obs['sample'].value_counts())

In [ ]:
# Checking the total size of the data set
adata.shape

Scanpy stores the count data is an annotated data matrix (observations e.g. cell barcodes × variables e.g gene names) called AnnData together with annotations of observations __(obs)__, variables __(var)__ and unstructured annotations __(uns)__

## Pre-processing and visualization

### Quality control

A high fraction of mitochondrial reads being picked up can indicate cell stress, as there is a low proportion of nuclear mRNA in the cell. It should be noted that high mitochondrial RNA fractions can also be biological signals indicating elevated respiration.

Typical quality measures for assessing the quality of a cell includes the following components

* Number of molecule counts (UMIs or n_counts )
* Number of expressed genes (n_genes)
* Fraction of counts that are mitochondrial (percent_mito)

We are calculating the above mentioned details using the following codes.

Cell barcodes with high count depth, few detected genes and high fraction of mitochondrial counts may indicate cells whose cytoplasmic mRNA has leaked out due to a broken membrane and only the mRNA located in the mitochondria has survived.

Cells with high UMI counts and detected genes may represent doublets (it requires further checking).

In [ ]:
# Quality control - calculate QC covariates
adata.obs['n_counts'] = adata.X.sum(1)
adata.obs['log_counts'] = np.log(adata.obs['n_counts'])
adata.obs['n_genes'] = (adata.X > 0).sum(1)

mt_gene_mask = [gene.startswith('mt-') for gene in adata.var_names]
adata.obs['mt_frac'] = adata.X[:, mt_gene_mask].sum(1)/adata.obs['n_counts']

In [ ]:
# Quality control - plot QC metrics
#Sample quality plots
t1 = sc.pl.violin(adata, 'n_counts', groupby='sample', size=2, log=True, cut=0)
t2 = sc.pl.violin(adata, 'mt_frac', groupby='sample')

In [ ]:
#Data quality summary plots
p1 = sc.pl.scatter(adata, 'n_counts', 'n_genes', color='mt_frac')
p2 = sc.pl.scatter(adata[adata.obs['n_counts']<10000], 'n_counts', 'n_genes', color='mt_frac')

In [ ]:
#Thresholding decision: counts
p3 = sb.distplot(adata.obs['n_counts'], kde=False)
plt.show()

p4 = sb.distplot(adata.obs['n_counts'][adata.obs['n_counts']<4000], kde=False, bins=60)
plt.show()

p5 = sb.distplot(adata.obs['n_counts'][adata.obs['n_counts']>10000], kde=False, bins=60)
plt.show()

In [ ]:
#Thresholding decision: genes
p6 = sb.distplot(adata.obs['n_genes'], kde=False, bins=60)
plt.show()

p7 = sb.distplot(adata.obs['n_genes'][adata.obs['n_genes']<1000], kde=False, bins=60)
plt.show()

In [ ]:
# Filter cells according to identified QC thresholds:
print('Total number of cells: {:d}'.format(adata.n_obs))

sc.pp.filter_cells(adata, min_counts = 1500)
print('Number of cells after min count filter: {:d}'.format(adata.n_obs))

sc.pp.filter_cells(adata, max_counts = 40000)
print('Number of cells after max count filter: {:d}'.format(adata.n_obs))

adata = adata[adata.obs['mt_frac'] < 0.2]
print('Number of cells after MT filter: {:d}'.format(adata.n_obs))

sc.pp.filter_cells(adata, min_genes = 700)
print('Number of cells after gene filter: {:d}'.format(adata.n_obs))

In [ ]:
#Filter genes:
print('Total number of genes: {:d}'.format(adata.n_vars))

# Min 20 cells - filters out 0 count genes
sc.pp.filter_genes(adata, min_cells=20)
print('Number of genes after cell filter: {:d}'.format(adata.n_vars))

### Normalization

Molecular counts in the matrix are coming from the mRNA molecules which are successfully captured, reverse transcribed and sequenced. Count depths for identical cells can be different due to the variability added by each of these above mentioned steps. The count data has to be normalized before the gene expressions can be compared between cells.

The most common normalization technique is count depth scaling (or Counts per million (CPM) normalization) which assumes that all the cells in the dataset initially contained an equal number of mRNA molecules. 

Typically, single cell data consists of heterogeneous cell populations where cell can have different sizes and molecular counts.

Scran ([Lun et al, 2016](https://genomebiology.biomedcentral.com/articles/10.1186/s13059-016-0947-7)) uses a pooling based size factor estimation method which can address this cell heterogenity issue.

The following code estimates a cell specific size factor (using scran package) and then divide the total count by that size factor for each cell.

This process requires a simple preprocessing step:
* Library size normalization to counts per millions (CPM)
* Log transformation
* Low resolution clustering


In [ ]:
#Perform a clustering for scran normalization in clusters
adata_pp = adata.copy()
sc.pp.normalize_per_cell(adata_pp, counts_per_cell_after=1e6)
sc.pp.log1p(adata_pp)
sc.pp.pca(adata_pp, n_comps=15)
sc.pp.neighbors(adata_pp)
sc.tl.louvain(adata_pp, key_added='groups', resolution=0.5)

In [ ]:
#Preprocess variables for scran normalization
input_groups = adata_pp.obs['groups']
data_mat = adata.X.T

In [ ]:
%%R -i data_mat -i input_groups -o size_factors

size_factors = computeSumFactors(data_mat, clusters=input_groups, min.mean=0.1)

In [ ]:
#Delete adata_pp
del adata_pp
del data_mat
del input_groups

We plot the size factors to show they are non-negative and related to the number of counts and genes per cell as expected.

In [ ]:
# Visualize the estimated size factors
adata.obs['size_factors'] = size_factors

sc.pl.scatter(adata, 'size_factors', 'n_counts')
sc.pl.scatter(adata, 'size_factors', 'n_genes')

sb.distplot(size_factors, bins=50, kde=False)
plt.show()

In [ ]:
#Keep the count data in a counts layer
adata.layers["counts"] = adata.X.copy()

Count matrix are typically log(x+1) transformed to convert expression values to log fold change which also reduces the skewness of the data (as the downstream tools assume the data is normally distributed)

In [ ]:
#Normalize adata 
adata.X /= adata.obs['size_factors'].values[:,None]
sc.pp.log1p(adata)

In [ ]:
# converting data to sparse matrix
import scipy
adata.X = scipy.sparse.csr_matrix(adata.X)

In [ ]:
# Store the full data set in 'raw' as log-normalised data for statistical testing
adata.raw = adata

### Batch Correction

Batch effect can be added to the data when they are handled in distinct batches or groups.
e.g. cells on the different chips or cells on the different sequencing lanes or cells harvested at different time point.

The best method of batch correction: Prevent batch effect by addressing it to the experiment design ([Hicks et al, 2017](https://pubmed.ncbi.nlm.nih.gov/29121214/))


Batch correction is performed to adjust for batch effects from the 3 samples that were loaded.

As the batch effect from samples and from epithelium regions are overlapping, correcting for this batch effect will also partially regress out differences between regions. We allow for this to optimally cluster the data. This approach can also be helpful to find differentiation trajectories, but we revert back to non-batch-corrected data for differential testing and computing marker genes.

Note that ComBat batch correction requires a dense matrix format as input (which is already the case in this example).

In [ ]:
# ComBat batch correction
sc.pp.combat(adata, key='sample')

### Highly Variable Genes

We extract highly variable genes (HVGs) to further reduce the dimensionality of the dataset and include only the most informative genes. Genes that vary substantially across the dataset are informative of the underlying biological variation in the data. As we only want to capture biological variation in these genes, we select highly variable genes after normalization and batch correction. HVGs are used for clustering, trajectory inference, and dimensionality reduction/visualization, while the full data set is used for computing marker genes, differential testing, cell cycle scoring, and visualizing expression values on the data.

Here we use a standard technique for the extraction of highly variable genes from the 10X genomics preprocessing software CellRanger. Typically between 1000 and 5000 genes are selected. Here, we extract the top 4000 most variable genes for further processing. If particular genes of importance are known, one could assess how many highly variable genes are necessary to include all, or the majority, of these.

In [ ]:
sc.pp.highly_variable_genes(adata, flavor='cell_ranger', n_top_genes=4000)
print('\n','Number of highly variable genes: {:d}'.format(np.sum(adata.var['highly_variable'])))

In [ ]:
sc.pl.highly_variable_genes(adata)

The plots show how the data was normalized to select highly variable genes irrespective of the mean expression of the genes. This is achieved by using the index of dispersion which divides by mean expression, and subsequently binning the data by mean expression and selecting the most variable genes within each bin.

Highly variable gene information is stored automatically in the adata.var['highly_variable'] field. The dataset now contains:

a 'counts' layer with count data
log-normalized data in adata.raw
batch corrected data in adata.X
highly variable gene annotations in adata.var['highly_variable']
The HVG labels will be used to subselect genes for clustering and trajectory analysis.

### Visualization

Visualizing scRNA-seq data is the process of projecting a high-dimensional matrix of cells and genes into a few coordinates such that every cell is meaningfully represented in a two-dimensional graph. However, the visualization of scRNA-seq data is an active area of research and each method defines 'meaningful' in its own way. Thus, it is a good idea to look at several visualizations and decide which visualization best represents the aspect of the data that is being investigated.

Overall t-SNE visualizations have been very popular in the community, however the recent UMAP algorithm has been shown to better represent the topology of the data.

Note that we do not scale the genes to have zero mean and unit variance. A lack of rescaling is equivalent to giving genes with a higher mean expression a higher weight in dimensionality reduction (despite correcting for mean offsets in PCA, due to the mean-variance relationship). We argue that this weighting based on mean expression being a biologically relevant signal. However, rescaling HVG expression is also common, and the number of publications that use this approach suggests that scaling is at least not detrimental to downstream scRNA-seq analysis.


In [ ]:
# Calculate the visualizations
sc.pp.pca(adata, n_comps=40, use_highly_variable=True, svd_solver='arpack')
sc.pp.neighbors(adata)

sc.tl.tsne(adata, n_jobs=2) #Note n_jobs works for MulticoreTSNE, but not regular implementation)
sc.tl.umap(adata)
sc.tl.diffmap(adata)
sc.tl.draw_graph(adata)

In [ ]:
sc.pl.pca_scatter(adata, color='n_counts')
sc.pl.tsne(adata, color='n_counts')
sc.pl.umap(adata, color='n_counts')
sc.pl.diffmap(adata, color='n_counts', components=['1,2','1,3'])
sc.pl.draw_graph(adata, color='n_counts')

__PCA__:

* Unsurprisingly, the first principle component captures variation in count depth between cells, and is thus only marginally informative
* The plot does not show the expected clustering of the data in two dimensions

__t-SNE__:

* Shows several distinct clusters with clear subcluster structure
* Connections between clusters are difficult to interpret visually

__UMAP__:

* Data points are spread out on the plot showing several clusters
*Connections between clusters can be readily identified


__Diffusion Maps__:

* Shows connections between regions of higher density
* Very clear trajectories are suggested, but clusters are less clear
* Each diffusion component extracts heterogeneity in a different part of the data

__Graph__:

* Shows a central cluster and several outer clusters
* Shows clear connections from the central cluster (likely stem cells) to outer clusters

The strengths and weaknesses of the visualizations can readily be identified in the above plots. While t-SNE exaggerates differences, diffusion maps exaggerate transitions. Overall UMAP and force-directed graph drawings show the best compromise of the two aspects, however UMAP is much faster to compute. UMAP has furthermore been shown to more accurately display the structure in the data.

### 3D UMAP plot

In [ ]:
sc.tl.leiden(adata)

In [ ]:
from copy import deepcopy
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
leiden_series = deepcopy(adata.obs['leiden'])
cell_clusters = list(leiden_series.value_counts().to_dict().keys())
colors = sc.pl.palettes.default_102[0:len(cell_clusters) ]
dict_map = dict(zip(cell_clusters,colors))
color_map = leiden_series.map(dict_map).values
labels = list(adata.obs.index)

sc.tl.umap(
  adata,
  n_components=3)
hovertext = \
  ['cluster: {0}, barcode: {1}'.\
     format(
       grp,labels[index])
         for index,grp in enumerate(leiden_series.values)]
## plotting 3D UMAP as html file
plot(
  [go.Scatter3d(
     x=adata.obsm['X_umap'][:, 0],
     y=adata.obsm['X_umap'][:, 1],
     z=adata.obsm['X_umap'][:, 2], 
     mode='markers',
     marker=dict(color=color_map,
                 size=5),
     opacity=0.6,
     text=labels,
     hovertext=hovertext,
  )],
  filename='UMAP-3D-plot.html')

In [ ]:
sc.tl.umap(adata)

In [ ]:
plt.rcParams['figure.figsize']=(11,9)
sc.pl.umap(adata, color=['leiden'],use_raw=False,palette=sc.pl.palettes.default_102)

In [ ]:
sc.pl.umap(adata, color=['sample'],use_raw=False)

### Cell cycle scoring

Known sources of technical variation in the data have been investigated and corrected for (e.g. batch, count depth). A known source of biological variation that can explain the data is the cell cycle. Here, a gene list from [Macosko et al., Cell 161 (2015)](https://www.cell.com/fulltext/S0092-8674(15)00549-8) is used to score the cell cycle effect in the data and classify cells by cell cycle phase. The file can be found on the single-cell-tutorial github repository, or be taken from the supplementary material of the paper.

Please note, that s the gene list was generated for human HeLa cells, the gene names are put into lower case with a leading capital letter to map to the respective mouse genes. When adapting this script to your own data, this must be taken into account for data from species other than mouse.

We perform cell cycle scoring on the full batch-corrected data set in adata.

In [ ]:
%%file /tmp/Macosko_cell_cycle_genes.txt
IG1.S,S,G2.M,M,M.G1
ACD,ABCC5,ANLN,AHI1, AGFG1
ACYP1,ABHD10,AP3D1,AKIRIN2,AGPAT3
ADAMTS1,ANKRD18A,ARHGAP19,ANKRD40,AKAP13
ANKRD10,ASF1B,ARL4A,ANLN,AMD1
APEX2,ATAD2,ARMC1,ANP32B,ANP32E
ARGLU1,BBS2,ASXL1,ANP32E,ANTXR1
ATAD2,BIVM,ATL2,ARHGAP19,BAG3
BARD1,BLM,AURKB,ARL6IP1,BTBD3
BRD7,BMI1,BCLAF1,ASXL1,CBX3
C1orf63,BRCA1,BORA,ATF7IP,CDC42
C7orf41,BRIP1,BRD8,AURKA,CDK7
C14orf142,C5orf42,BUB3,BIRC2,CDKN3
CAPN7,C11orf82,C2orf69,BIRC5,CEP70
CASP2,CALD1,C14orf80,BUB1,CNIH4
CASP8AP2,CALM2,CASP3,CADM1,CTR9
CCNE1,CASP2,CBX5,CCDC88A,CWC15
CCNE2,CCDC14,CCDC107,CCDC90B,DCP1A
CDC6,CCDC84,CCNA2,CCNA2,DCTN6
CDC25A,CCDC150,CCNF,CCNB2,DEXI
CDCA7,CDC7,CDC16,CDC20,DKC1
CDCA7L,CDC45,CDC25C,CDC25B,DNAJB6
CEP57,CDCA5,CDCA2,CDC27,DSP
CHAF1A,CDKN2AIP,CDCA3,CDC42EP1,DYNLL1
CHAF1B,CENPM,CDCA8,CDCA3,EIF4E
CLSPN,CENPQ,CDK1,CENPA,ELP3
CREBZF,CERS6,CDKN1B,CENPE,FAM60A
CTSD,CHML,CDKN2C,CENPF,FAM189B
DIS3,COQ9,CDR2,CEP55,FOPNL
DNAJC3,CPNE8,CENPL,CFLAR,FOXK2
DONSON,CREBZF,CEP350,CIT,FXR1
DSCC1,CRLS1,CFD,CKAP2,G3BP1
DTL,DCAF16,CFLAR,CKAP5,GATA2
E2F1,DEPDC7,CHEK2,CKS1B,GNB1
EIF2A,DHFR,CKAP2,CKS2,GRPEL1
ESD,DNA2,CKAP2L,CNOT10,GSPT1
FAM105B,DNAJB4,CYTH2,CNTROB,GTF3C4
FAM122A,DONSON,DCAF7,CTCF,HIF1A
FLAD1,DSCC1,DHX8,CTNNA1,HMG20B
GINS2,DYNC1LI2,DNAJB1,CTNND1,HMGCR
GINS3,E2F8,ENTPD5,DEPDC1,HSD17B11
GMNN,EIF4EBP2,ESPL1,DEPDC1B,HSPA8
HELLS,ENOSF1,FADD,DIAPH3,ILF2
HOXB4,ESCO2,FAM83D,DLGAP5,JMJD1C
HRAS,EXO1,FAN1,DNAJA1,KDM5B
HSF2,EZH2,FANCD2,DNAJB1,KIAA0586
INSR,FAM178A,G2E3,DR1,KIF5B
INTS8,FANCA,GABPB1,DZIP3,KPNB1
IVNS1ABP,FANCI,GAS1,E2F5,KRAS
KIAA1147,FEN1,GAS2L3,ECT2,LARP1
KIAA1586,GCLM,H2AFX,FAM64A,LARP7
LNPEP,GOLGA8A,HAUS8,FOXM1,LRIF1
LUC7L3,GOLGA8B,HINT3,FYN,LYAR
MCM2,H1F0,HIPK2,G2E3,MORF4L2
MCM4,HELLS,HJURP,GADD45A,MRPL19
MCM5,HIST1H2AC,HMGB2,GAS2L3,MRPS2
MCM6,HIST1H4C,HN1,GOT1,MRPS18B
MDM1,INTS7,HP1BP3,GRK6,MSL1
MED31,KAT2A,HRSP12,GTSE1,MTPN
MRI1,KAT2B,IFNAR1,HCFC1,NCOA3
MSH2,KDELC1,IQGAP3,HMG20B,NFIA
NASP,KIAA1598,KATNA1,HMGB3,NFIC
NEAT1,LMO4,KCTD9,HMMR,NUCKS1
NKTR,LYRM7,KDM4A,HN1,NUFIP2
NPAT,MAN1A2,KIAA1524,HP1BP3,NUP37
NUP43,MAP3K2,KIF5B,HPS4,ODF2
ORC1,MASTL,KIF11,HS2ST1,OPN3
OSBPL6,MBD4,KIF20B,HSPA8,PAK1IP1
PANK2,MCM8,KIF22,HSPA13,PBK
PCDH7,MLF1IP,KIF23,INADL,PCF11
PCNA,MYCBP2,KIFC1,KIF2C,PLIN3
PLCXD1,NAB1,KLF6,KIF5B,PPP2CA
PMS1,NEAT1,KPNA2,KIF14,PPP2R2A
PNN,NFE2L2,LBR,KIF20B,PPP6R3
POLD3,NRD1,LIX1L,KLF9,PRC1
RAB23,NSUN3,LMNB1,LBR,PSEN1
RECQL4,NT5DC1,MAD2L1,LMNA,PTMS
RMI2,NUP160,MALAT1,MCM4,PTTG1
RNF113A,OGT,MELK,MDC1,RAD21
RNPC3,ORC3,MGAT2,MIS18BP1,RAN
SEC62,OSGIN2,MID1,MKI67,RHEB
SKP2,PHIP,MIS18BP1,MLLT4,RPL13A
SLBP,PHTF1,MND1,MZT1,SLC39A10
SLC25A36,PHTF2,NCAPD3,NCAPD2,SNUPN
SNHG10,PKMYT1,NCAPH,NCOA5,SRSF3
SRSF7,POLA1,NCOA5,NEK2,STAG1
SSR3,PRIM1,NDC80,NUF2,SYNCRIP
TAF15,PTAR1,NEIL3,NUP35,TAF9
TIPIN,RAD18,NFIC,NUP98,TCERG1
TOPBP1,RAD51,NIPBL,NUSAP1,TLE3
TRA2A,RAD51AP1,NMB,ODF2,TMEM138
TTC14,RBBP8,NR3C1,ORAOV1,TOB2
UBR7,REEP1,NUCKS1,PBK,TOP1
UHRF1,RFC2,NUMA1,PCF11,TROAP
UNG,RHOBTB3,NUSAP1,PLK1,TSC22D1
USP53,RMI1,PIF1,POC1A,TULP4
VPS72,RPA2,PKNOX1,POM121,UBE2D3
WDR76,RRM1,POLQ,PPP1R10,VANGL1
ZMYND19,RRM2,PPP1R2,PRPSAP1,VCL
ZNF367,RSRC2,PSMD11,PRR11,WIPF2
ZRANB2,SAP30BP,PSRC1,PSMG3,WWC1
,SLC38A2,RANGAP1,PTP4A1,YY1
,SP1,RCCD1,PTPN9,ZBTB7A
,SRSF5,RDH11,PWP1,ZCCHC10
,SVIP,RNF141,QRICH1,ZNF24
,TOP2A,SAP30,RAD51C,ZNF281
,TTC31,SKA3,RANGAP1,ZNF593
,TTLL7,SMC4,RBM8A,
,TYMS,STAT1,RCAN1,
,UBE2T,STIL,RERE,
,UBL3,STK17B,RNF126,
,USP1,SUCLG2,RNF141,
,ZBED5,TFAP2A,RNPS1,
,ZWINT,TIMP1,RRP1,
,,TMEM99,SEPHS1,
,,TMPO,SETD8,
,,TNPO2,SFPQ,
,,TOP2A,SGOL2,
,,TRAIP,SHCBP1,
,,TRIM59,SMARCB1,
,,TRMT2A,SMARCD1,
,,TTF2,SPAG5,
,,TUBA1A,SPTBN1,
,,TUBB,SRF,
,,TUBB2A,SRSF3,
,,TUBB4B,SS18,
,,TUBD1,SUV420H1,
,,UACA,TACC3,
,,UBE2C,THRAP3,
,,VPS25,TLE3,
,,VTA1,TMEM138,
,,WSB1,TNPO1,
,,ZNF587,TOMM34,
,,ZNHIT2,TPX2,
,,,TRIP13,
,,,TSG101,
,,,TSN,
,,,TTK,
,,,TUBB4B,
,,,TXNDC9,
,,,TXNRD1,
,,,UBE2D3,
,,,USP13,
,,,USP16,
,,,VANGL1,
,,,WIBG,
,,,WSB1,
,,,YWHAH,
,,,ZC3HC1,
,,,ZFX,
,,,ZMYM1,
,,,ZNF207,

In [ ]:
#Score cell cycle and visualize the effect:
cc_genes = pd.read_csv(cc_genes_file, sep=',')
s_genes = cc_genes['S'].dropna()
g2m_genes = cc_genes['G2.M'].dropna()

s_genes_mm = [gene.lower().capitalize() for gene in s_genes]
g2m_genes_mm = [gene.lower().capitalize() for gene in g2m_genes]

s_genes_mm_ens = adata.var_names[np.in1d(adata.var_names, s_genes_mm)]
g2m_genes_mm_ens = adata.var_names[np.in1d(adata.var_names, g2m_genes_mm)]

sc.tl.score_genes_cell_cycle(adata, s_genes=s_genes_mm_ens, g2m_genes=g2m_genes_mm_ens)

In [ ]:
sc.pl.umap(adata, color='S_score', use_raw=False)
sc.pl.umap(adata, color='G2M_score', use_raw=False)
sc.pl.umap(adata, color='phase', use_raw=False)

Cell cycle scoring shows a pronounced proliferation signal in the largest central cluster in the Umap representation. This suggests a proliferation phenotype in these cells. Other clusters also show less pronouced cell cycle structure.

## Downstream analysis

### Clustering

Clustering is a central component of the scRNA-seq analysis pipeline. To understand the data, we must identify cell types and states present. The first step of doing so is clustering. Performing Modularity optimization by Louvain community detection on the k-nearest-neighbour graph of cells has become an established practice in scRNA-seq analysis. Thus, this is the method of choice in this tutorial as well.

Here, we perform clustering at two resolutions. Investigating several resolutions allows us to select a clustering that appears to capture the main clusters in the visualization and can provide a good baseline for further subclustering of the data to identify more specific substructure.

Clustering is performed on the highly variable gene data, dimensionality reduced by PCA, and embedded into a KNN graph. (see `sc.pp.pca()` and `sc.pp.neighbors()` functions used in the visualization section.

In [ ]:
# Perform clustering - using highly variable genes
sc.tl.louvain(adata, key_added='louvain_r1')
sc.tl.louvain(adata, resolution=0.5, key_added='louvain_r0.5', random_state=10)

In [ ]:
adata.obs['louvain_r1'].value_counts()

In [ ]:
adata.obs['louvain_r0.5'].value_counts()

In [ ]:
adata.obs['leiden'].value_counts()

In [ ]:
sc.tl.louvain(adata, resolution=0.6, key_added='louvain_r0.6', random_state=10)

In [ ]:
adata.obs['louvain_r0.6'].value_counts()

In [ ]:
sc.tl.leiden(adata, resolution=0.6, key_added='leiden_r0.6', random_state=10)

In [ ]:
adata.obs['leiden_r0.6'].value_counts()

In [ ]:
#Visualize the clustering and how this is reflected by different technical covariates
sc.pl.umap(adata, color='louvain_r1', palette=sc.pl.palettes.default_102)
sc.pl.umap(adata, color='louvain_r0.5', palette=sc.pl.palettes.default_102)
sc.pl.umap(adata, color='louvain_r0.6', palette=sc.pl.palettes.default_102)
sc.pl.umap(adata, color='leiden', palette=sc.pl.palettes.default_102)
sc.pl.umap(adata, color='leiden_r0.6', palette=sc.pl.palettes.default_102)
sc.pl.umap(adata, color=['region', 'n_counts'])
sc.pl.umap(adata, color=['log_counts', 'mt_frac'])

__Note__: Due to memory (RAM) limitation, we have used only 3 samples and then adjusted the parameters to match the plots from the original study.

At a resolution of 0.6 the broad clusters in the visualization are captured well in the data. The covariate plots show that clusters 0 and 6 in this data set are characterized by low and high counts respectively. In the case of cluster 6 this may be biologically relevant, while cluster 0 is also characterized by higher mitochondrial read fractions. This indicates cell stress.

The variation captured by the clustering is not related to the intestinal epithelium regions. This suggests we are detecting cell types across regions as intended.

### Marker genes & cluster annotation

To annotate the clusters we obtained, we find genes that are up-regulated in the cluster compared to all other clusters (marker genes). This differential expression test is performed by a Welch t-test with overestimated variance to be conservative. This is the default in scanpy. The test is automatically performed on the `.raw` data set, which is uncorrected and contains all genes. All genes are taken into account, as any gene may be an informative marker.

As we are using the relevant mouse gut atlas from the literature in this case study, there is no other reference atlas which we use to annotate the cells by automated annotation. Thus, we do not use scmap or garnett here.

In [ ]:
#Calculate marker genes
sc.tl.rank_genes_groups(adata, groupby='leiden_r0.6', key_added='rank_genes_leiden_r0.6')

In [ ]:
#Plot marker genes
sc.pl.rank_genes_groups(adata, key='rank_genes_leiden_r0.6',ncols=2)

To further identify the clusters in our data set, we look at the overlap with a list of known marker genes. Here, these marker genes were taken from the publication from which the data was obtained ([Haber et al, 2018](https://www.nature.com/articles/nature24489)).

In practice marker gene sets can be obtained from public databases such as Linnarson's mouse brain atlas, various Human Cell Atlas datasets, and other published reference atlases. It should be noted that marker genes may not always overlap as expected given that atlases tend to be investigated under wild-type conditions.

In [ ]:
#Known marker genes:
marker_genes = dict()
marker_genes['Stem'] = ['Lgr5', 'Ascl2', 'Slc12a2', 'Axin2', 'Olfm4', 'Gkn3']
marker_genes['Enterocyte (Proximal)'] = ['Gsta1','Rbp2','Adh6a','Apoa4','Reg3a','Creb3l3','Cyp3a13','Cyp2d26','Ms4a10','Ace','Aldh1a1','Rdh7','H2-Q2', 'Hsd17b6','Gstm3','Gda','Apoc3','Gpd1','Fabp1','Slc5a1','Mme','Cox7a1','Gsta4','Lct','Khk','Mttp','Xdh','Sult1b1', 'Treh','Lpgat1','Dhrs1','Cyp2c66','Ephx2','Cyp2c65','Cyp3a25','Slc2a2','Ugdh','Gstm6','Retsat','Ppap2a','Acsl5', 'Cyb5r3','Cyb5b','Ckmt1','Aldob','Ckb','Scp2','Prap1']
marker_genes['Enterocyte (Distal)'] = ['Tmigd1','Fabp6','Slc51b','Slc51a','Mep1a','Fam151a','Naaladl1','Slc34a2','Plb1','Nudt4','Dpep1','Pmp22','Xpnpep2','Muc3','Neu1','Clec2h','Phgr1','2200002D01Rik','Prss30','Cubn','Plec','Fgf15','Crip1','Krt20','Dhcr24','Myo15b','Amn','Enpep','Anpep','Slc7a9','Ocm','Anxa2','Aoc1','Ceacam20','Arf6','Abcb1a','Xpnpep1','Vnn1','Cndp2','Nostrin','Slc13a1','Aspa','Maf','Myh14']
marker_genes['Goblet'] = ['Agr2', 'Fcgbp', 'Tff3', 'Clca1', 'Zg16', 'Tpsg1', 'Muc2', 'Galnt12', 'Atoh1', 'Rep15', 'S100a6', 'Pdia5', 'Klk1', 'Pla2g10', 'Spdef', 'Lrrc26', 'Ccl9', 'Bace2', 'Bcas1', 'Slc12a8', 'Smim14', 'Tspan13', 'Txndc5', 'Creb3l4', 'C1galt1c1', 'Creb3l1', 'Qsox1', 'Guca2a', 'Scin', 'Ern2', 'AW112010', 'Fkbp11', 'Capn9', 'Stard3nl', 'Slc50a1', 'Sdf2l1', 'Hgfa', 'Galnt7', 'Hpd', 'Ttc39a', 'Tmed3', 'Pdia6', 'Uap1', 'Gcnt3', 'Tnfaip8', 'Dnajc10', 'Ergic1', 'Tsta3', 'Kdelr3', 'Foxa3', 'Tpd52', 'Tmed9', 'Spink4', 'Nans', 'Cmtm7', 'Creld2', 'Tm9sf3', 'Wars', 'Smim6', 'Manf', 'Oit1', 'Tram1', 'Kdelr2', 'Xbp1', 'Serp1', 'Vimp', 'Guk1', 'Sh3bgrl3', 'Cmpk1', 'Tmsb10', 'Dap', 'Ostc', 'Ssr4', 'Sec61b', 'Pdia3', 'Gale', 'Klf4', 'Krtcap2', 'Arf4', 'Sep15', 'Ssr2', 'Ramp1', 'Calr', 'Ddost']
marker_genes['Paneth'] = ['Gm15284', 'AY761184', 'Defa17', 'Gm14851', 'Defa22', 'Defa-rs1', 'Defa3', 'Defa24', 'Defa26', 'Defa21', 'Lyz1', 'Gm15292', 'Mptx2', 'Ang4']
marker_genes['Enteroendocrine'] = ['Chgb', 'Gfra3', 'Cck', 'Vwa5b2', 'Neurod1', 'Fev', 'Aplp1', 'Scgn', 'Neurog3', 'Resp18', 'Trp53i11', 'Bex2', 'Rph3al', 'Scg5', 'Pcsk1', 'Isl1', 'Maged1', 'Fabp5', 'Celf3', 'Pcsk1n', 'Fam183b', 'Prnp', 'Tac1', 'Gpx3', 'Cplx2', 'Nkx2-2', 'Olfm1', 'Vim', 'Rimbp2', 'Anxa6', 'Scg3', 'Ngfrap1', 'Insm1', 'Gng4', 'Pax6', 'Cnot6l', 'Cacna2d1', 'Tox3', 'Slc39a2', 'Riiad1']
marker_genes['Tuft'] = ['Alox5ap', 'Lrmp', 'Hck', 'Avil', 'Rgs13', 'Ltc4s', 'Trpm5', 'Dclk1', 'Spib', 'Fyb', 'Ptpn6', 'Matk', 'Snrnp25', 'Sh2d7', 'Ly6g6f', 'Kctd12', '1810046K07Rik', 'Hpgds', 'Tuba1a', 'Pik3r5', 'Vav1', 'Tspan6', 'Skap2', 'Pygl', 'Ccdc109b', 'Ccdc28b', 'Plcg2', 'Ly6g6d', 'Alox5', 'Pou2f3', 'Gng13', 'Bmx', 'Ptpn18', 'Nebl', 'Limd2', 'Pea15a', 'Tmem176a', 'Smpx', 'Itpr2', 'Il13ra1', 'Siglecf', 'Ffar3', 'Rac2', 'Hmx2', 'Bpgm', 'Inpp5j', 'Ptgs1', 'Aldh2', 'Pik3cg', 'Cd24a', 'Ethe1', 'Inpp5d', 'Krt23', 'Gprc5c', 'Reep5', 'Csk', 'Bcl2l14', 'Tmem141', 'Coprs', 'Tmem176b', '1110007C09Rik', 'Ildr1', 'Galk1', 'Zfp428', 'Rgs2', 'Inpp5b', 'Gnai2', 'Pla2g4a', 'Acot7', 'Rbm38', 'Gga2', 'Myo1b', 'Adh1', 'Bub3', 'Sec14l1', 'Asah1', 'Ppp3ca', 'Agt', 'Gimap1', 'Krt18', 'Pim3', '2210016L21Rik', 'Tmem9', 'Lima1', 'Fam221a', 'Nt5c3', 'Atp2a3', 'Mlip', 'Vdac3', 'Ccdc23', 'Tmem45b', 'Cd47', 'Lect2', 'Pla2g16', 'Mocs2', 'Arpc5', 'Ndufaf3']

In [ ]:
cell_annotation = sc.tl.marker_gene_overlap(adata, marker_genes, key='rank_genes_leiden_r0.6')
cell_annotation

We can also visualize the marker gene overlap as a fraction of the total marker genes, and then plot this as a heatmap for simpler cell identity annotation.

In [ ]:
cell_annotation_norm = sc.tl.marker_gene_overlap(adata, marker_genes, key='rank_genes_leiden_r0.6', normalize='reference')
sb.heatmap(cell_annotation_norm, cbar=False, annot=True)

Here we look simply at the fraction of known marker genes that are found in the cluster marker gene sets from the rank_genes_groups() function. This allows us to clearly identify tuft cells, enteroendocrine cells, paneth cells, enterocytes, and stem cells.

A more rigorous analysis would be to perform an enrichment test. Yet, in this data set the assignment is sufficiently clear so that it is not necessary.

As we see quite an overlap of Goblet markers in cluster 6, which otherwise seems to contain paneth cells, we will visualize the expression of two markers to show the respective populations.

Note that use_raw=False is used to visualize batch-corrected data on top of the UMAP layout.

In [ ]:
{
    'TA':0, # 1
    'EP (early)':1, # 0 
    'Stem':2, # 2
    'Goblet':3, #3
    'EP (stress)':4, # 6
    'Enterocyte':5, # 4
    'Paneth':6, # 5
    'Enteroendocrine':7,#8
    'Tuft':8}#7

In [ ]:
['EP (early)','TA','Stem','Goblet','Enterocyte','Paneth','EP (stress)','Tuft','Enteroendocrine']

In [ ]:
from matplotlib import colors
#Define a nice colour map for gene expression
colors2 = plt.cm.Reds(np.linspace(0, 1, 128))
colors3 = plt.cm.Greys_r(np.linspace(0.7,0.8,20))
colorsComb = np.vstack([colors3, colors2])
mymap = colors.LinearSegmentedColormap.from_list('my_colormap', colorsComb)

In [ ]:
#Defa24 #Tff3
sc.pl.umap(adata, color='Defa24', use_raw=False, color_map=mymap)
sc.pl.umap(adata, color='Tff3', use_raw=False, color_map=mymap)

It is evident that the two clusters are distinct, yet goblet cell markers may also be expressed in paneth cells at a lower level.

To identify clusters 0, 1, and 6, we now look at known marker gene expression. It is possible that a known marker is expressed in a cluster although it is not a marker gene for this cluster given its expression in another cluster is higher. This can be the case especially for progenitor cells. We visualize gene expression on the full, batch-corrected data set in adata.

Given the position of clusters 0, 1, and 6 in the UMAP visualization enterocyte and stem cell markers are of particular interest.

In [ ]:
# Check expression of enterocyte markers
#Collate all enterocyte markers and get the gene IDs in the data set
ids_entprox = np.in1d(adata.var_names, marker_genes['Enterocyte (Proximal)'])
ids_entdist = np.in1d(adata.var_names, marker_genes['Enterocyte (Distal)'])
ids_ent = np.logical_or(ids_entprox, ids_entdist)

#Calculate the mean expression of enterocyte markers
adata.obs['Enterocyte_marker_expr'] = adata.X[:,ids_ent].mean(1)

#Plot enterocyte expression
sc.pl.violin(adata, 'Enterocyte_marker_expr', groupby='leiden_r0.6')
sc.pl.umap(adata, color='Enterocyte_marker_expr', color_map=mymap)

In [ ]:
#Early enterocyte marker - Arg2
sc.pl.umap(adata, color='Arg2', use_raw=False, color_map=mymap)

sc.pl.violin(adata, groupby='leiden_r0.6', keys='Arg2', use_raw=False)

sc.pl.diffmap(adata, components=['6,9'], color='Arg2', use_raw=False, color_map=mymap)
sc.pl.diffmap(adata, components=['6,9'], color='leiden_r0.6')

The violin plots show that enterocyte marker expression is slightly higher in clusters 1 and 4 compared to other clusters except for the enterocyte cluster 5. For cluster 4 this is particular noticeable in Arg2 expression, which is an enterocyte marker that can already be measured at early stages of differentiation.

The diffusion map with Arg2 expression visualized confirms that clusters 1 and 4 are between stem cells and enterocytes and are positioned where Arg2 expression is present. Cluster 0, on the other hand seems to be a little separated from the enterocyte trajectory.

In [ ]:
sc.pl.violin(adata, 'mt_frac', groupby='leiden_r0.6')
sc.pl.violin(adata, 'log_counts', groupby='leiden_r0.6')

We looked at technical covariates to confirm the 'stressed' phenotype of cluster 4 to differentiate clusters 1 and 4.

In [ ]:

#Check individual stem markers
stem_genes = adata.var_names[np.in1d(adata.var_names, marker_genes['Stem'])]
sc.pl.umap(adata, color=stem_genes[:3], title=stem_genes[:3], color_map=mymap)
sc.pl.umap(adata, color=stem_genes[3:], title=stem_genes[3:], color_map=mymap)

In [ ]:
adata.obs['Stem_marker_expr'] = adata[:,stem_genes].X.mean(1)

sc.pl.violin(adata, 'Stem_marker_expr', groupby='leiden_r0.6')
sc.pl.umap(adata, color='Stem_marker_expr', color_map=mymap)

Clusters 0 and 1 show more stem-like expression patterns compared to clusters 4. Together with the cell-cycle signature seen in a previous plot, cluster 0 shows a proliferative signature, and thus points towards transit amplifying cells.

Clusters 1 and 4 have heightened enterocyte markers, but are more stem-like than enterocytes. They likely consist of enterocyte progenitors (EPs).

Cluster 1 appears to represent an earlier stage of EP than cluster 4 given the diffusion map, and the enterocyte marker expression levels. It may consist of a mixture of Stem cells and EPs.

The identified cell types are renamed in the full data set.

In [ ]:
#Categories to rename
adata.obs['leiden_r0.6'].cat.categories

In [ ]:
adata.rename_categories('leiden_r0.6',['EP (early)','TA','Stem','Goblet','Enterocyte','Paneth','EP (stress)','Tuft','Enteroendocrine'] )

In [ ]:
adata.obs['leiden_r0.6'].value_counts()

In [ ]:
plt.rcParams['figure.figsize']=(10,9)
sc.pl.umap(adata, color='leiden_r0.6', size=15, legend_loc='on data',palette=sc.pl.palettes.default_102)

### Subclustering

To build on the basis clustering, we can now subcluster parts of the data to identify substructure within the identified cell types. Here, we subcluster the 'Enterocyte' population to see if we can find distal and proximal enterocyte clusters which were obtained in the (Haber et al. 2018) paper.

Subclustering is normally performed at a lower resolution than on the entire dataset given that clustering is more sensitive when performed on a small subset of the data.

In [ ]:
#Subcluster enterocytes
sc.tl.louvain(adata, restrict_to=('leiden_r0.6', ['Enterocyte']), resolution=0.2, key_added='leiden_r0.6_entero_sub')

In [ ]:
#Show the new clustering
if 'leiden_r0.6_entero_sub_colors' in adata.uns:
    del adata.uns['leiden_r0.6_entero_sub_colors']

sc.pl.umap(adata, color='leiden_r0.6_entero_sub', palette=sc.pl.palettes.default_102)
sc.pl.umap(adata, color='region', palette=sc.pl.palettes.default_102)

The subclustering has identified four subclusters of enterocytes. Plots of the the intestinal regions show that proximal (duodenum and jejunum) and distal (ileum) enterocytes have been separated in some clusters.

Marker genes are now computed to verify this observation quantitatively.

In [ ]:
#Get the new marker genes
sc.tl.rank_genes_groups(adata, groupby='leiden_r0.6_entero_sub', key_added='rank_genes_r0.6_entero_sub')

In [ ]:
#Plot the new marker genes
sc.pl.rank_genes_groups(adata, key='rank_genes_r0.6_entero_sub', groups=['Enterocyte,0','Enterocyte,1','Enterocyte,2'], ncols=2, fontsize=12)

To visualize that the markers genes we detect are indeed more highly expressed in our cluster compared to background gene expression, we will now plot the last 10 marker genes (numbers 91-100) that we detect per cluster.

We do this to check that there are indeed at least 100 valid marker genes for each cluster, and we don't just detect noise.

In [ ]:
entero_clusts = [clust for clust in adata.obs['leiden_r0.6_entero_sub'].cat.categories if clust.startswith('Enterocyte')]

for clust in entero_clusts:
    sc.pl.rank_genes_groups_violin(adata, use_raw=True, key='rank_genes_r0.6_entero_sub', groups=[clust], gene_names=adata.uns['rank_genes_r0.6_entero_sub']['names'][clust][90:100])

These genes appear up-regulated in our cluster. We will now test for overlap with known distal and proximal markers, and assess how strong the enterocyte markers are expressed in the subclusters.

In [ ]:
#Subset marker gene dictionary to only check for enterocyte markers
marker_genes_entero = {k: marker_genes[k] for k in marker_genes if k.startswith('Enterocyte')}

In [ ]:
#Find marker overlap
sc.tl.marker_gene_overlap(adata, marker_genes_entero, key='rank_genes_r0.6_entero_sub', normalize='reference')

In [ ]:
#Check enterocyte marker expression
sc.pl.violin(adata[adata.obs['leiden_r0.6']=='Enterocyte'], groupby='leiden_r0.6_entero_sub', keys='Enterocyte_marker_expr')

In [ ]:
#Visualize some enterocyte markers
entero_genes = ['Alpi', 'Apoa1', 'Apoa4', 'Fabp1', 'Arg2']
sc.pl.umap(adata, color=entero_genes[:3], title=entero_genes[:3], color_map=mymap)
sc.pl.umap(adata, color=entero_genes[3:], title=entero_genes[3:], color_map=mymap)

In [ ]:
sc.pl.diffmap(adata, color='leiden_r0.6_entero_sub', components='3,7')

Marker gene expression and overlap show that Enterocyte cluster 0 contains distal, and cluster 1 contains proximal enterocytes. Total enterocyte marker expression in the violin plot identifies clusters 0 and 1 as immature distal and immature proximal enterocytes respectively, while the 'Enterocyte,2' cluster contains mature enterocytes from both proximal and distal populations.

Assuming that the diffusion map and Umap reprentations show a differentiation trajectory from stem cells to enterocytes, this provides further support for our labelling of immature and mature enterocyte populations.

In [ ]:
tmp = adata.obs['leiden_r0.6_entero_sub'].cat.categories

tmp = ['Enterocyte imm. (Distal)' if item == 'Enterocyte,0' else item for item in tmp]
tmp = ['Enterocyte imm. (Proximal)' if item == 'Enterocyte,2' else item for item in tmp]
tmp = ['Enterocyte mature' if item == 'Enterocyte,1' else item for item in tmp]


adata.rename_categories('leiden_r0.6_entero_sub', tmp)

To see if we can separate mature enterocytes further into proximal and distal regions, we can iteratively subcluster.

In [ ]:
#Subcluster mature enterocytes
sc.tl.louvain(adata, restrict_to=('leiden_r0.6_entero_sub', ['Enterocyte mature']), resolution=0.25, key_added='leiden_r0.6_entero_mat_sub')

In [ ]:
#Show the new clustering
if 'leiden_r0.6_entero_mat_sub_colors' in adata.uns:
    del adata.uns['leiden_r0.6_entero_mat_sub_colors']

sc.pl.umap(adata, color='leiden_r0.6_entero_mat_sub', palette=sc.pl.palettes.default_102)

In [ ]:
#Get the new marker genes
sc.tl.rank_genes_groups(adata, groupby='leiden_r0.6_entero_mat_sub', key_added='rank_genes_r0.6_entero_mat_sub')

In [ ]:
#Plot the new marker genes
sc.pl.rank_genes_groups(adata, key='rank_genes_r0.6_entero_mat_sub',
                        groups=['Enterocyte mature,0','Enterocyte mature,1'], fontsize=12)

In [ ]:
pd.DataFrame(adata.uns['rank_genes_r0.6_entero_mat_sub']['names'])['Enteroendocrine'].head(15).to_list()

In [ ]:
entero_mat_clusts = [clust for clust in adata.obs['leiden_r0.6_entero_mat_sub'].cat.categories \
                       if clust.startswith('Enterocyte mature')]

for clust in entero_mat_clusts:
    sc.pl.rank_genes_groups_violin(
        adata, use_raw=True, key='rank_genes_r0.6_entero_mat_sub', groups=[clust], 
        gene_names=adata.uns['rank_genes_r0.6_entero_mat_sub']['names'][clust][90:100])

In [ ]:
#Find marker overlap
sc.tl.marker_gene_overlap(adata, marker_genes_entero, key='rank_genes_r0.6_entero_mat_sub', normalize='reference')

This separation of mature enterocytes has worked to a certain extent based on marker gene overlap. 'Enterocyte mature,0' appear to be distal mature enterocytes and 'Enterocyte mature,1' appear to be more proximal mature enterocytes (although more mixed than the distal cluster).

It gets increasingly difficult to evaluate separated distal and proximal enterocytes based on marker genes. It appears that mature enterocytes share more otherwise distal and proximal markers than immature or intermediate enterocytes. A further complication is that we have partially removed the differences in the proximal and distal enterocyte populations via batch correction. This explains why clustering is not separating enterocytes between proximal and distal populations as well.

In [ ]:
tmp = adata.obs['leiden_r0.6_entero_mat_sub'].cat.categories

tmp = ['Enterocyte mat. (Distal)' if item == 'Enterocyte mature,0' else item for item in tmp]
tmp = ['Enterocyte mat. (Proximal)' if item == 'Enterocyte mature,1' else item for item in tmp]

adata.rename_categories('leiden_r0.6_entero_mat_sub', tmp)

In [ ]:
adata.obs['leiden_final'] = adata.obs['leiden_r0.6_entero_mat_sub']

In [ ]:
sc.pl.umap(adata, color='leiden_final', palette=sc.pl.palettes.default_102, legend_loc='on data',legend_fontsize=8)

In [ ]:
adata.obs['leiden_final'].value_counts()

### Compositional analysis

While it is not straightforward to test whether cell-type compositions have change between conditions (see main paper), we can visualize shifts in cellular densities between conditions. Here we visualize the densities of distal and proximal intestinal cells.

In [ ]:
#Define a variable that stores proximal and distal labels
adata.obs['prox_dist'] = ['Distal' if reg=='Il' else 'Proximal' for reg in adata.obs['region']]

In [ ]:
sc.tl.embedding_density(adata, basis='umap', groupby='prox_dist')

In [ ]:
adata.obs['prox_dist'].value_counts()
sc.pl.embedding_density(adata, basis='umap', key='umap_density_prox_dist', group='Proximal')
sc.pl.embedding_density(adata, basis='umap', key='umap_density_prox_dist', group='Distal')

It appears that proximal intestinal cells have higher proportions of stem cells, enterocyte progenitors, and transit amplifying cells, while distal intestinal cells have high proportions of enterocytes and goblet cells. Although this analysis was not performed in the publication (Haber et al., Nature 2018), the latter observation is supported by the literature (Barker, van de Wetering, and Clevers, Genes & Development 2008).

We should note that only relative proportions can be visually compared. The number of cells in each sample should not be taken into account as this is a parameter which is not indicative of absolute cell numbers in the intestinal epithelium, but rather related to the experimental design.

### Trajectory inference and pseudotime analysis

As our data set contains differentiation processes, we can investigate the differentiation trajectories in the data. This analysis is centred around the concept of 'pseudotime'. In pseudotime analysis a latent variable is inferred based on which the cells are ordered. This latent variable is supposed to measure the differentiation state along a trajectory.

Pseudotime analysis is complicated when there are multiple trajectories in the data. In this case, the trajectory structure in the data must first be found before pseudotime can be inferred along each trajectory. The analysis is then called 'trajectory inference'.

Once the pseudotime variable is inferred, we can test for genes that vary continuously along pseudotime. These genes are seen as being associated with the trajectory, and may play a regulatory role in the potential differentiation trajectory that the analysis found.

Here, we measure the trajectory from stem cells to enterocytes, which was also studied in the Haber et al. paper. We also investigate which genes vary along pseudotime.

Based on a recent comparison of pseudotime methods [Saelens et al., 2018], we have selected the top performing 'Slingshot', 'Monocle2', and 'Diffusion Pseudotime (DPT)'. Three methods were chosen as trajectory inference is a complex problem which is not yet solved. Different methods perform well on different types of trajectories. For example, 'Slingshot' was the top performer for simple bifurcating and multifurcating trajectories; 'Monocle2' performed best for complex tree structures, and 'DPT' performed well in bifurcating trajectories and was used in the Haber et al paper from which we took this dataset. As the complexity of trajectories are generally not known, it is adviseable to compare trajectory inference outputs.

In [ ]:
adata.obs['leiden_final'].value_counts()

We first subset the data to include only Stem cells, Enterocyte Progenitor cells (EP), Transit Amplifying cells (TA), and the Enterocyte subclusters. After subsetting it is important to recalculate the dimensionality reduction methods such as PCA, and diffusion maps, as the variability of the subsetted data set will be projected onto different basis vectors.

Note that we subset the data to include only HVGs. Trajectory inference, and especially measuring gene expression changes over pseudotime can be a computationally expensive process, thus we often work with reduced gene sets that are informative of the variance in the data, such as HVGs.

In [ ]:
#Subsetting to relevant clusters
clusters_to_include = [g for g in adata.obs['leiden_final'].cat.categories \
                         if (g.startswith('Enterocyte') or \
                             g.startswith('TA') or \
                             g.startswith('Stem') or \
                             g.startswith('EP'))]
adata_ent = adata[np.isin(adata.obs['leiden_final'], clusters_to_include),:].copy()

#Subset to highly variable genes
sc.pp.highly_variable_genes(adata_ent, flavor='cell_ranger', n_top_genes=4000, subset=True)

As we have subsetted the data to include only cell types that we assume are of interest, we recalculate the dimension reduction algorithms on this data. This ensures that for example the first few PCs capture only the variance in this data and not variance in parts of the full data set we have filtered out.

In [ ]:
#Recalculating PCA for subset
sc.pp.pca(adata_ent, svd_solver='arpack')
sc.pl.pca(adata_ent)
sc.pl.pca_variance_ratio(adata_ent)

Trajectory inference is often performed on PCA-reduced data, as is the case for Slingshot and Monocle2. To assess how many principal components (PCs) should be included in the low-dimensional representation we can use the 'elbow method'. This method involves looking for the 'elbow' in the plot of the variance ratio explained per PC. Above we can see the elbow at PC7. Thus the first seven PCs are included in the slingshot data.

In [ ]:
adata_ent.obsm['X_pca'] = adata_ent.obsm['X_pca'][:,0:7]

### Slingshot

Slingshot is written in R. The integration of R in this notebook is again achieved via the rpy2 interface. We use a specifically developed package called anndata2ri (https://www.github.com/flying-sheep/anndata2ri), that takes care of the conversion from an AnnData object to SingleCellExperiment object in R. It should be noted that the convention for scRNA-seq data matrix storage in R is opposite to python. In R the expression matrix is stored as genes x cells rather than cells x genes. Thus, the matrix must be transposed before being input into the R function. This is already taken care of by anndata2ri.

We are loading the normalized, log-transformed, and batch-corrected data as we want to minimize technical variation in the inferred trajectories.

Implementation note:

this section closely follows the online Slingshot tutorial

In [ ]:
%%R -i adata_ent

#Plot 1
colour_map = brewer.pal(20,'Set1')
par(xpd=TRUE)
par(mar=c(4.5,5.5,2,7))
plot(reducedDims(adata_ent)$PCA[,1], reducedDims(adata_ent)$PCA[,2], col=colour_map[colData(adata_ent)$leiden_final], bty='L', xlab='PC1', ylab='PC2')
legend(x=12, y=12, legend=unique(colData(adata_ent)$leiden_final), fill=colour_map[as.integer(unique(colData(adata_ent)$leiden_final))])

print("1:")
adata_ent_start <- slingshot(adata_ent, clusterLabels = 'leiden_final', reducedDim = 'PCA', start.clus='Stem')
print(SlingshotDataSet(adata_ent_start))

print("")
print("2:")
adata_ent_startend <- slingshot(adata_ent, clusterLabels = 'leiden_final', reducedDim = 'PCA', start.clus='Stem', end.clus=c('Enterocyte mat. (Proximal)', 'Enterocyte mat. (Distal)'))
print(SlingshotDataSet(adata_ent_startend))

print("")
print("3:")
adata_ent_simple_startend <- slingshot(adata_ent, clusterLabels = 'leiden_r0.6', reducedDim = 'PCA', start.clus='Stem', end.clus='Enterocyte')
print(SlingshotDataSet(adata_ent_simple_startend))

In [ ]:
%%R
options(repr.plot.width=10, repr.plot.height=12)
plot(reducedDims(adata_ent)$PCA[,1], reducedDims(adata_ent)$PCA[,2], col=colour_map[colData(adata_ent)$leiden_final], bty='L', xlab='PC1', ylab='PC2')
legend(x=12, y=12, legend=unique(colData(adata_ent)$leiden_final), fill=colour_map[as.integer(unique(colData(adata_ent)$leiden_final))])


Here we output three inferred sets of trajectories and a plot of how the data look on a two principal component representation. The plot shows that the differentiation from stem to enterocytes is broadly captured within the first two PCs. However, it is not clear whether proximal and distal enterocyte fates are separated.

The inferred trajectories can be seen in the 'lineages:' output. In the first trajectory, no endpoints are fixed and only the 'Stem' cell compartment is fixed as a starting point; the second trajectory includes fixed mature proximal and distal enterocytes as endpoints; and the third trajectory is performed over the simpler clustering without enterocyte subtypes.

The inferred trajectories show that when no endpoints are fixed, the detected lineage does not distinguish between proximal and distal enterocyte endpoints. It then looks similar to the inferred trajectory without enterocyte subgroups. Trajectory inference with fixed endpoints vastly improves the trajectory and only shows an overlap of immature proximal and distal enterocytes in the distal lineage. This can be easily explained when looking at the PCA plot. In the first two PCs immature proximal and distal enterocytes overlap fully.

Furthermore, TA cells cannot be fit into the enterocyte differentiation trajectory in any method. A reason for this may be that the cell-cycle effect is affecting the trajectory inference algorithm. A cell-cycle correction algorithm such as scLVM or simply regressing out the cell cycle may remove this issue. Another possible explanation is that the TA cell cluster is more involved in differentiation towards other cell fates that we have filtered out.

The above trajectories can be visualized with Slingshot custom visualization tools.

Implementation note:

In the next step we don't have to input adata_ent, adata_ent_startend, or adata_ent_simple_startend, as these are still available from the computation in code cell 79. In fact, as adata_ent_startend is not a SingleCellExperiment object, but a SlingshotObject, data will be lost when outputting this object back into python with anndata2ri.

In [ ]:
%%R

#Plot of lineage 1
colors <- colorRampPalette(brewer.pal(11,'Spectral')[-6])(100)
plot(reducedDims(adata_ent_startend)$PCA[,c(1,2)], col = colors[cut(adata_ent_startend$slingPseudotime_1,breaks=100)], pch=16, asp = 1, xlab='PC1', ylab='PC2')
lines(slingCurves(adata_ent_startend)$curve1, lwd=2)

#Plot of lineage 2
colors <- colorRampPalette(brewer.pal(11,'Spectral')[-6])(100)
plot(reducedDims(adata_ent_startend)$PCA[,c(1,2)], col = colors[cut(adata_ent_startend$slingPseudotime_2,breaks=100)], pch=16, asp = 1, xlab='PC1', ylab='PC2')
lines(slingCurves(adata_ent_startend)$curve2, lwd=2)

#Plot of lineages with clusters visualized
par(xpd=TRUE)
plot(reducedDims(adata_ent_startend)$PCA[,c(1,2)], col = brewer.pal(11,'Set1')[adata_ent$leiden_final], pch=16, asp = 1, bty='L', xlab='PC1', ylab='PC2')
lines(SlingshotDataSet(adata_ent_startend), lwd=2, type='lineages')
legend(x=10, y=20, legend=unique(colData(adata_ent)$leiden_final), fill=brewer.pal(11,'Set1')[as.integer(unique(colData(adata_ent)$leiden_final))])

#Plot of simpler clustering
plot(reducedDims(adata_ent_simple_startend)$PCA[,c(1,2)], col = colors[cut(adata_ent_simple_startend$slingPseudotime_1,breaks=100)], pch=16, asp = 1, xlab='PC1', ylab='PC2')
lines(SlingshotDataSet(adata_ent_simple_startend), lwd=2)

These plots show the lineages inferred by Slingshot. The first three plots show the lineages inferred when fixing start and end points, and the final plot shows the lineages on the clustering without enterocyte subclusters. The first, second, and fourth plots are coloured by the first pseudotime variable in the first or second lineages. As TA cells are not in these inferred lineages, TA cells are not shown in these plots.

We can see that the immature enterocyte cluster centres overlap on the first two PCs which explains the difficulty of separating the two lineages. Furthermore, the placement of the 'EP (stress)' and 'TA' clusters in the second plot suggests that PC2 may have captured cell cycle variability, as these clusters show a stronger cell cycle signature than other clusters.

Note that the performance of Slingshot will rely on the clustering that was input. For example, while the expected trajectory from stem cells to enterocytes is found with coarse clustering, higher resolution subclusters that resolve proximal and distal enterocyte populations only partially render clearly separated proximal and distal enterocyte trajectories. This can have several reasons including batch over-correction removing the biological difference; a lack of resolution in precursor states that do not allow for higher resolution trajectories; or simply an insufficient difference in the transcriptome between proximal and distal enterocytes.

Furthermore, as mentioned above cell cycle is informative for cluster identification, however it may interfere with trajectory analysis.

### Diffusion Pseudotime (DPT)

Finally, we include Diffusion Pseudotime in the analysis to further support the found trajectories. Diffusion pseudotime is integrated into scanpy and is therefore easy to use with the current setup.

DPT is based on diffusion maps, thus a diffusion map representation must be calculated prior to pseudotime inference. This in turn is based on a KNN graph embedding obtained from sc.pp.neighbors().

In [ ]:
sc.pp.neighbors(adata_ent)
sc.tl.diffmap(adata_ent)

In [ ]:
sc.pl.diffmap(adata_ent, components='1,2', color='leiden_final')
sc.pl.diffmap(adata_ent, components='1,3', color='leiden_final')

Looking at the first three diffusion components (DCs) we can see that DC3 separates the proximal and distal enterocyte trajectories.

In DPT we must assign a root cell to infer pseudotime. In the plots we can observe that the most appropriate root will be the Stem cell with the minimum DC1, or DC3 value, or the maximum DC2 value.

Note that 'DC3' is stored in adata_ent.obsm['X_diffmap'][:,3] as the 0-th column is the steady-state solution, which is non-informative in diffusion maps.

In [ ]:
#Find the stem cell with the highest DC3 value to act as root for the diffusion pseudotime and compute DPT
stem_mask = np.isin(adata_ent.obs['leiden_final'], 'Stem')
max_stem_id = np.argmin(adata_ent.obsm['X_diffmap'][stem_mask,3])
root_id = np.arange(len(stem_mask))[stem_mask][max_stem_id]
adata_ent.uns['iroot'] = root_id

#Compute dpt
sc.tl.dpt(adata_ent)

In [ ]:
#Visualize pseudotime over differentiation
sc.pl.diffmap(adata_ent, components='1,3', color='dpt_pseudotime')

## Gene expression dynamics

To find genes that describe the differentiation process, we can investigate how gene expression varies across pseudotime in different trajectories. Essentially, we smooth the pseudotime variable and the expression profile of each gene, and fit a curve to the data. In our case, a generalized additive model (gam) performs well.

It should be noted that this calculation is the most computationally expensive part of the entire workflow by quite a distance. This process can take up to an hour on a single core depending on computational load. This is approximately half of the time of the entire script.

In [ ]:
%%R

#Set the pseudotime variable
t <- adata_ent_simple_startend$slingPseudotime_1

#Extract the gene expression matrix
Y <- assay(adata_ent_simple_startend)

# fit a GAM with a loess term for pseudotime
#Note: This takes a while
gam.pval <- apply(Y,1,function(z){
  d <- data.frame(z=z, t=t)
  tmp <- gam(z ~ lo(t), data=d)
  p <- summary(tmp)[4][[1]][1,5]
  p
})

In [ ]:
%%R -w 600 -h 1200

#Select the top 100 most significant genes that change over pseudotime
topgenes <- names(sort(gam.pval, decreasing = FALSE))[1:100]
heatdata <- assay(adata_ent_simple_startend)[rownames(assay(adata_ent_simple_startend)) %in% topgenes, 
                        order(t, na.last = NA)]

#Scale the data per gene for visualization
heatdata <- t(scale(t(heatdata)))

#Trimm z-score scale
heatdata[heatdata > 3] = 3
heatdata[heatdata < -3] = -3

#Get cluster assignment
heatclus <- adata_ent_simple_startend$leiden_r0.6[order(t, na.last = NA)]

#Set up a clusterExperiment data structure for heatmap visualization
ce <- ClusterExperiment(heatdata, heatclus, transformation = function(x){x})

#Plot the heatmap
plotHeatmap(ce, clusterSamplesData = "orderSamplesValue", visualizeData = 'transformed', fontsize=15)


The above plot shows nicely how the gene expression dynamics change over pseudotime. Further, we can also see that the clusters are not entirely separated over pseudotime (from the bar above the plot). This is especially visible between EP (early) and EP (stress), which is expected given the two clusters are both marked as enterocyte progenitors.

In the visualization it should be noted that the absolute expression levels can no longer be compared between genes given the z-score scaling. However, we can see at which points genes are turned on along pseudotime.

To better interpret the above plot we look for overlaps between genes that change with pseudotime and known enterocyte marker genes.

## Partition-based graph abstraction

Partition-based graph abstraction (PAGA) is a method to reconcile clustering and pseudotemporal ordering. It can be applied to an entire dataset and does not assume that there are continuous trajectories connecting all clusters.

As PAGA is integrated into scanpy, we can easily run it on the entire data set. Here we run and visualize PAGA with different clustering inputs.

In [ ]:
sc.tl.paga(adata, groups='leiden_r0.6')

In [ ]:
adata.uns.get('paga')

In [ ]:
sc.pl.paga(adata, color='leiden_r0.6',cmap=sc.pl.palettes.default_102)
sc.pl.umap(adata, color='leiden_r0.6', palette=sc.pl.palettes.default_102)

In [ ]:
sc.tl.draw_graph(adata, init_pos='paga')

In [ ]:
sc.pl.draw_graph(adata, color='leiden_r0.6',palette=sc.pl.palettes.default_102)

In [ ]:
plt.rcParams['figure.figsize']=(8,8)
sc.pl.paga_compare(adata, basis='umap')

We can do the same visualization on a umap layout.

In [ ]:
fig1, ax1 = plt.subplots()
sc.pl.umap(adata, size=50, ax=ax1,show=False)
sc.pl.paga(adata, pos=adata.uns['paga']['pos'],show=False,  node_size_scale=2, node_size_power=1, ax=ax1, text_kwds={'alpha':0})

Implementation note:

Note that the above plotting function only works when sc.pl.paga_compare(adata, basis='umap') is run before. The sc.pl.paga_compare() function stores the correct positions in adata.uns['paga']['pos'] to overlay the PAGA plot with a umap representation. To overlap PAGA with other representation, you can run sc.pl.paga_compare() with other basis parameters before plotting the combined plot.
Using the simpler clustering, the PAGA plot shows the expected transition between stem cells to enterocytes that traverses the EP and/or TA cells. Interestingly TA cells are also included meaningfully in this trajectory, although not as expected directly from Stem cells. This is likely because the 'EP (early)' cluster includes stem cells as well as early enterocyte progenitors. Indeed, the connectivities of the 'TA', 'EP (early)' and 'Stem' clusters with other clusters in the dataset indicate that these may all contain stem cells.

Furthermore, regressing out the cell cycle effect will likely change how 'TA' cells are included in the trajectory. In this manner trajectory inference and graph abstraction can be iteratively improved.

A noteworthy result is the separation of absorptative (enterocyte) and secretory (tuft, paneth, enteroendocrine, and goblet) lineages in the intestine. Further iterative improvement can be applied to the secretory lineage region of the graph. For example. the UMAP representation shows a transition between paneth and stem cells which we expect to occur in the data. Paneth cells have more counts per cell than stem cells which can complicate the trajectory inference.

## Gene-level analysis

### Gene set analysis

For the paneth data set we have no literature-derived markers that distinguish distal and proximal regions. Thus, we show a typical analysis approach to understand differential expression signals: functional enrichment. Specifically, we are looking for Gene Ontology Biological Process annotations that are enriched in the set of differentially expressed genes between proximal and distal paneth cells.

This analysis is performed with g:profiler, or specifically with Valentine Svensson's python API for the g:profiler webtool. It should be noted that g:profiler's python API is currently being redesigned and thus the old python API (used here) is no longer supported and updated. To use the newest GO data, please check with the g:profiler team.

In [ ]:
gp = GProfiler(return_dataframe=True, user_agent='g:GOSt')

In [ ]:
endo_genes = ['Fam183b',
 'Gfra3',
 'Aplp1',
 'Chgb',
 'Tm4sf4',
 'Cdkn1a',
 'Gch1',
 'Marcksl1',
 'Ddc',
 'Cpe',
 'Cystm1',
 'Fxyd3',
 'Ngfrap1',
 'Fev',
 'Cryba2']

In [ ]:
paneth_enrichment = gp.profile(organism='mmusculus', sources=['GO:BP'], user_threshold=0.05,
                               significance_threshold_method='fdr', 
                               background=adata.var_names.tolist(), 
                               query=endo_genes)

In [ ]:
paneth_enrich_results = paneth_enrichment.set_index('native').sort_values('p_value').iloc[:,[2,5,7,10,1]]

In [ ]:
pd.set_option("display.max_colwidth", 800)
paneth_enrich_results.iloc[:50,:]

Here we find differential expression of immune response GO terms enriched between proximal and distal paneth cells (eg. "defense response to bacterium", "antimicrobial humoral response", or "defense response to other organism"). As paneth cells are involved in innate immune response, these findings are consistent with prior knowledge. Differences between the proximal and distal regions are likely to be found within representative functions of the cell type.

It is common to do this analysis with only up- or down-regulated genes depending on which biological hypotheses are investigated. By performing GO term enrichment on the up- and down-regulated genes together we find annotations of biological processes that are differentially expressed between proximal and distal paneth cells, but we cannot say whether one of the two regions particularly up-regulates the genes in a process or down-regulates them. This can only be addressed by separating genes based on log fold change.

It should be noted that relative over- or under-expression of genes does not have to map to up- or down-regulation of a biological process. For example, relatively lower expression levels of a pathway inhibitor, will result in an up-regulation of the activity of this pathway, while the opposite is true for low expression of a promoter. Thus, a more detailed investigation is generally necessary to conclude the direction of regulation of a process between conditions.

We can display the above results in a more appealing way using a dotplot. This plotting scheme is similar to the one in the clusterProfiler R package.

In [ ]:
## codes copied from https://github.com/theislab/single-cell-tutorial

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
from matplotlib import colors
from matplotlib import rcParams


def scale_data_5_75(data):
    mind = np.min(data)
    maxd = np.max(data)
    
    if maxd == mind:
        maxd=maxd+1
        mind=mind-1
        
    drange = maxd - mind
    return ((((data - mind)/drange*0.70)+0.05)*100)


def plot_enrich(data, n_terms=20, save=False):
    # Test data input
    if not isinstance(data, pd.DataFrame):
        raise ValueError('Please input a Pandas Dataframe output by gprofiler.')
        
    if not np.all([term in data.columns for term in ['p_value', 'name', 'intersection_size']]):
        raise TypeError('The data frame {} does not contain enrichment results from gprofiler.'.format(data))
    
    data_to_plot = data.iloc[:n_terms,:].copy()
    data_to_plot['go.id'] = data_to_plot.index

    min_pval = data_to_plot['p_value'].min()
    max_pval = data_to_plot['p_value'].max()
    
    # Scale intersection_size to be between 5 and 75 for plotting
    #Note: this is done as calibration was done for values between 5 and 75
    data_to_plot['scaled.overlap'] = scale_data_5_75(data_to_plot['intersection_size'])
    
    norm = colors.LogNorm(min_pval, max_pval)
    sm = plt.cm.ScalarMappable(cmap="cool", norm=norm)
    sm.set_array([])

    rcParams.update({'font.size': 14, 'font.weight': 'bold'})

    sb.set(style="whitegrid")

    path = plt.scatter(x='recall', y="name", c='p_value', cmap='cool', 
                       norm=colors.LogNorm(min_pval, max_pval), 
                       data=data_to_plot, linewidth=1, edgecolor="grey", 
                       s=[(i+10)**1.5 for i in data_to_plot['scaled.overlap']])
    ax = plt.gca()
    ax.invert_yaxis()

    ax.set_ylabel('')
    ax.set_xlabel('Gene ratio', fontsize=14, fontweight='bold')
    ax.xaxis.grid(False)
    ax.yaxis.grid(True)

    # Shrink current axis by 20%
    box = ax.get_position()
    ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])

    # Get tick marks for this plot
    #Note: 6 ticks maximum
    min_tick = np.floor(np.log10(min_pval)).astype(int)
    max_tick = np.ceil(np.log10(max_pval)).astype(int)
    tick_step = np.ceil((max_tick - min_tick)/6).astype(int)
    
    # Ensure no 0 values
    if tick_step == 0:
        tick_step = 1
        min_tick = max_tick-1
    
    ticks_vals = [10**i for i in range(max_tick, min_tick-1, -tick_step)]
    ticks_labs = ['$10^{'+str(i)+'}$' for i in range(max_tick, min_tick-1, -tick_step)]

    #Colorbar
    fig = plt.gcf()
    cbaxes = fig.add_axes([0.8, 0.15, 0.03, 0.4])
    cbar = ax.figure.colorbar(sm, ticks=ticks_vals, shrink=0.5, anchor=(0,0.1), cax=cbaxes)
    cbar.ax.set_yticklabels(ticks_labs)
    cbar.set_label("Adjusted p-value", fontsize=14, fontweight='bold')

    #Size legend
    min_olap = data_to_plot['intersection_size'].min()
    max_olap = data_to_plot['intersection_size'].max()
    olap_range = max_olap - min_olap
    
    #Note: approximate scaled 5, 25, 50, 75 values are calculated
    #      and then rounded to nearest number divisible by 5
    size_leg_vals = [np.round(i/5)*5 for i in 
                          [min_olap, min_olap+(20/70)*olap_range, min_olap+(45/70)*olap_range, max_olap]]
    size_leg_scaled_vals = scale_data_5_75(size_leg_vals)

    
    l1 = plt.scatter([],[], s=(size_leg_scaled_vals[0]+10)**1.5, edgecolors='none', color='black')
    l2 = plt.scatter([],[], s=(size_leg_scaled_vals[1]+10)**1.5, edgecolors='none', color='black')
    l3 = plt.scatter([],[], s=(size_leg_scaled_vals[2]+10)**1.5, edgecolors='none', color='black')
    l4 = plt.scatter([],[], s=(size_leg_scaled_vals[3]+10)**1.5, edgecolors='none', color='black')

    labels = [str(int(i)) for i in size_leg_vals]

    leg = plt.legend([l1, l2, l3, l4], labels, ncol=1, frameon=False, fontsize=12,
                     handlelength=1, loc = 'center left', borderpad = 1, labelspacing = 1.4,
                     handletextpad=2, title='Gene overlap', scatterpoints = 1,  bbox_to_anchor=(-2, 1.5), 
                     facecolor='black')

    if save:
        plt.savefig(save, dpi=300, format='pdf')

    plt.show()

In [ ]:
plot_enrich(paneth_enrich_results)

In the above plot the number of differentially expressed genes (DEGs) that share a particular GO term is represented by the size of the datapoint, the color shows the FDR-corrected p-value, and the x-axis is the ratio of genes in the full DEG set that contain the particular annotation.

## Explore cells in UCSC Cell Browser

The UCSC Cell Browser is an interactive browser for single cell data. You can visualize the PCA, t-SNA and UMAP plot of these cells using it. For more details, please check [Cellbrowser docs](https://cellbrowser.readthedocs.io/).

In [ ]:
sc.external.exporting.cellbrowser(
    adata,
    data_name='GSE92332',
    annot_keys=['leiden', 'percent_mito', 'n_genes', 'n_counts'],
    data_dir='/tmp/ucsc-data',
    cluster_field='leiden')

If you are using Binder for running this tutorial, please run next two cells (after removing the `#` from the `#!cbBuild -i...`) to access the UCSC Cell Browser.

In [ ]:
import os
if 'BINDER_LAUNCH_HOST' in os.environ:
  from IPython.display import HTML
  url = '<a href="{0}user/{1}/proxy/8080/">Cellbrowser UI</a>'.\
          format(
            os.environ.get('JUPYTERHUB_BASE_URL'),
            os.environ.get('JUPYTERHUB_CLIENT_ID').replace('jupyterhub-user-','')
          )
else:
  url = '<a href="http://localhost:8080/">Cellbrowser UI</a>'

HTML(url)

In [ ]:
#!cbBuild -i /tmp/ucsc-data/cellbrowser.conf -o /tmp/ucsc-tmp -p 8080 2> /dev/null

When you are done, feel free to stop the above cell by clicking on the stop button from the tab menu.

## References
* [Scanpy - Preprocessing and clustering 3k PBMCs](https://scanpy-tutorials.readthedocs.io/en/latest/pbmc3k.html)
* [single-cell-tutorial](https://github.com/theislab/single-cell-tutorial)

## Acknowledgement
The Imperial BRC Genomics Facility is supported by NIHR funding to the Imperial Biomedical Research Centre.
